In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import matplotlib.dates as mdates

Portfolio = r"C:\Users\Asus\OneDrive\Desktop\Project_Som\Data\Portfolio.xlsx"
optimal_portfolio_data = pd.read_excel(Portfolio, 'Optimal Portfolio')
optimal_portfolio_data['Timestamp'] = pd.to_datetime(optimal_portfolio_data['Timestamp'])
optimal_portfolio_data = optimal_portfolio_data.sort_values(by='Timestamp').reset_index(drop=True)

In [ ]:
optimal_portfolio_data.head()

In [3]:
stressed_periods = {
    "2008 Global Financial Crisis": ("2007-07-01", "2009-06-30"),
    "Taper Tantrum": ("2013-05-01", "2013-12-31"),
    "Demonetization in India": ("2016-11-01", "2017-01-31"),
    "GST Implementation": ("2017-07-01", "2017-12-31"),
    "IL&FS Crisis": ("2018-09-01", "2019-03-31"),
    "COVID-19 Outbreak": ("2020-02-01", "2020-12-31"),
    "Russia-Ukraine War": ("2022-02-01", "2022-12-31"),
}

def calculate_var_es(data, confidence_level=0.95):
    sorted_returns = np.sort(data)
    var_index = int((1 - confidence_level) * len(sorted_returns))
    var = sorted_returns[var_index]
    es = sorted_returns[:var_index].mean()
    return var, es

def plot_histogram(data, title, xlabel, ylabel):
    plt.hist(data, bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
results = {}

for period_name, (start_date, end_date) in stressed_periods.items():
    

    pre_stress_start = pd.to_datetime(start_date) - pd.DateOffset(months=6)
    pre_stress_end = pd.to_datetime(start_date)
    
    pre_stress_data = optimal_portfolio_data[
        (optimal_portfolio_data['Timestamp'] >= pre_stress_start) &
        (optimal_portfolio_data['Timestamp'] < pre_stress_end)
    ]['Portfolio'].pct_change().dropna()
    
    stress_data = optimal_portfolio_data[
        (optimal_portfolio_data['Timestamp'] >= start_date) &
        (optimal_portfolio_data['Timestamp'] <= end_date)
    ]['Portfolio'].pct_change().dropna()

    pre_stress_var, pre_stress_es = calculate_var_es(pre_stress_data, confidence_level=0.95)
    stress_var, stress_es = calculate_var_es(stress_data, confidence_level=0.95)
    

    rolling_window = 30
    pre_stress_rolling_var = pre_stress_data.rolling(window=rolling_window).apply(lambda x: np.percentile(x, 5), raw=True)
    stress_rolling_var = stress_data.rolling(window=rolling_window).apply(lambda x: np.percentile(x, 5), raw=True)
    
    results[period_name] = {
        "Pre-Stress VaR": pre_stress_var,
        "Pre-Stress ES": pre_stress_es,
        "Stress VaR": stress_var,
        "Stress ES": stress_es,
        "Pre-Stress Data": pre_stress_data,
        "Stress Data": stress_data,
    }
    
    plot_histogram(pre_stress_data, f"{period_name}: Pre-Stress Returns", "Returns", "Frequency")
    plot_histogram(stress_data, f"{period_name}: Stress Returns", "Returns", "Frequency")
    
    plt.figure(figsize=(12, 6))

    plt.plot(pre_stress_rolling_var, label="Pre-Stress Rolling VaR", color="blue")
    plt.plot(stress_rolling_var, label="Stress Rolling VaR", color="red")

    plt.axvspan(pre_stress_data.index[0], pre_stress_data.index[-1], color='blue', alpha=0.2, label="Pre-Stress Period")
    plt.axvspan(stress_data.index[0], stress_data.index[-1], color='orange', alpha=0.2, label="Stress Period")
    plt.gca().set_xticklabels([])
    plt.title(f"{period_name}: Rolling VaR", fontsize=16)
    plt.xlabel("Time")
    plt.ylabel("VaR")
    plt.legend()

    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
for period, metrics in results.items():
    print(f"{period}:")
    print(f"  Pre-Stress VaR: {metrics['Pre-Stress VaR']:.2%}, Pre-Stress ES: {metrics['Pre-Stress ES']:.2%}")
    print(f"  Stress VaR: {metrics['Stress VaR']:.2%}, Stress ES: {metrics['Stress ES']:.2%}")
    print()


In [5]:
def calculate_hybrid_var_es(data, confidence_level=0.95, window=30):
    rolling_vol = data.rolling(window=window).std()
    current_vol = rolling_vol.iloc[-1]
    scaled_returns = data * (current_vol / rolling_vol)
    scaled_returns = scaled_returns.dropna()
    sorted_returns = np.sort(scaled_returns)
    var_index = int((1 - confidence_level) * len(sorted_returns))
    var = sorted_returns[var_index]
    es = sorted_returns[:var_index].mean()

    return var, es, scaled_returns

In [ ]:
results_hhs = {}
window_size = 30

for period_name, (start_date, end_date) in stressed_periods.items():
    pre_stress_start = pd.to_datetime(start_date) - pd.DateOffset(months=6)
    pre_stress_end = pd.to_datetime(start_date)
    
    pre_stress_data = optimal_portfolio_data[
        (optimal_portfolio_data['Timestamp'] >= pre_stress_start) &
        (optimal_portfolio_data['Timestamp'] < pre_stress_end)
    ]['Portfolio'].pct_change().dropna()
    
    stress_data = optimal_portfolio_data[
        (optimal_portfolio_data['Timestamp'] >= start_date) &
        (optimal_portfolio_data['Timestamp'] <= end_date)
    ]['Portfolio'].pct_change().dropna()
    
    pre_stress_var, pre_stress_es, pre_stress_scaled = calculate_hybrid_var_es(pre_stress_data, confidence_level=0.95, window=window_size)
    stress_var, stress_es, stress_scaled = calculate_hybrid_var_es(stress_data, confidence_level=0.95, window=window_size)

    results_hhs[period_name] = {
        "Pre-Stress VaR": pre_stress_var,
        "Pre-Stress ES": pre_stress_es,
        "Stress VaR": stress_var,
        "Stress ES": stress_es,
        "Pre-Stress Scaled Returns": pre_stress_scaled,
        "Stress Scaled Returns": stress_scaled,
    }

    plot_histogram(pre_stress_scaled, f"{period_name}: Pre-Stress Returns", "Returns", "Frequency")
    plot_histogram(stress_scaled, f"{period_name}: Stress Returns", "Returns", "Frequency")

    plt.figure(figsize=(12, 6))
    plt.plot(pre_stress_scaled, label="Pre-Stress Scaled VaR", color="blue")
    plt.plot(stress_scaled, label="Stress Scaled VaR", color="red")
    plt.axvspan(pre_stress_data.index[0], pre_stress_data.index[-1], color='blue', alpha=0.2, label="Pre-Stress Period")
    plt.axvspan(stress_data.index[0], stress_data.index[-1], color='orange', alpha=0.2, label="Stress Period")
    plt.gca().set_xticklabels([])
    plt.title(f"{period_name}: Rolling VaR", fontsize=16)
    plt.xlabel("Time")
    plt.ylabel("VaR")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    print("HHS Analysis Results:")
for period, metrics in results_hhs.items():
    print(f"{period}:")
    print(f"  Pre-Stress VaR (HHS): {metrics['Pre-Stress VaR']:.2%}, Pre-Stress ES (HHS): {metrics['Pre-Stress ES']:.2%}")
    print(f"  Stress VaR (HHS): {metrics['Stress VaR']:.2%}, Stress ES (HHS): {metrics['Stress ES']:.2%}")
    print()

In [7]:
comparison_results = []

for period_name, metrics in results_hhs.items():
    traditional_pre_var = results[period_name]["Pre-Stress VaR"]
    traditional_pre_es = results[period_name]["Pre-Stress ES"]
    traditional_stress_var = results[period_name]["Stress VaR"]
    traditional_stress_es = results[period_name]["Stress ES"]

    hhs_pre_var = metrics["Pre-Stress VaR"]
    hhs_pre_es = metrics["Pre-Stress ES"]
    hhs_stress_var = metrics["Stress VaR"]
    hhs_stress_es = metrics["Stress ES"]

    var_diff_pre = abs((hhs_pre_var - traditional_pre_var)) 
    es_diff_pre = abs((hhs_pre_es - traditional_pre_es)) 
    var_diff_stress = abs((hhs_stress_var - traditional_stress_var)) 
    es_diff_stress = abs((hhs_stress_es - traditional_stress_es)) 

    comparison_results.append({
        "Period": period_name,
        "Pre-Stress Traditional VaR": traditional_pre_var,
        "Pre-Stress HHS VaR": hhs_pre_var,
        "Pre-Stress VaR Diff (%)": var_diff_pre,
        "Pre-Stress Traditional ES": traditional_pre_es,
        "Pre-Stress HHS ES": hhs_pre_es,
        "Pre-Stress ES Diff (%)": es_diff_pre,
        "Stress Traditional VaR": traditional_stress_var,
        "Stress HHS VaR": hhs_stress_var,
        "Stress VaR Diff (%)": var_diff_stress,
        "Stress Traditional ES": traditional_stress_es,
        "Stress HHS ES": hhs_stress_es,
        "Stress ES Diff (%)": es_diff_stress,
    })

In [ ]:
comparison_df = pd.DataFrame(comparison_results)

def plot_comparison_grouped(df, metric, title, ylabel):
    """
    Plot a grouped bar chart for comparing Traditional HS and HHS.
    """
    plt.figure(figsize=(14, 7))
    
    # Define positions for bars
    x = np.arange(len(df["Period"]))
    width = 0.35  # Width of each bar

    # Values for Traditional HS and HHS
    traditional = df[f"Pre-Stress Traditional {metric}"]
    hhs = df[f"Pre-Stress HHS {metric}"]

    # Create grouped bar chart
    plt.bar(x - width/2, traditional, width, label="Traditional HS", color='blue', alpha=0.7)
    plt.bar(x + width/2, hhs, width, label="HHS", color='red', alpha=0.7)

    # Add labels and title
    plt.title(title, fontsize=16)
    plt.xlabel("Stress Period", fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.xticks(x, df["Period"], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add value labels on the bars
    for i, v in enumerate(traditional):
        plt.text(i - width/2, v + 0.001, f"{v:.2%}", ha='center', va='bottom', fontsize=9)
    for i, v in enumerate(hhs):
        plt.text(i + width/2, v + 0.001, f"{v:.2%}", ha='center', va='bottom', fontsize=9)

    plt.show()


plot_comparison_grouped(comparison_df, "VaR", "Pre-Stress VaR: Traditional HS vs. HHS", "VaR")

plot_comparison_grouped(comparison_df, "VaR", "Stress Period VaR: Traditional HS vs. HHS", "VaR")

plot_comparison_grouped(comparison_df, "ES", "Pre-Stress ES: Traditional HS vs. HHS", "ES")

plot_comparison_grouped(comparison_df, "ES", "Stress Period ES: Traditional HS vs. HHS", "ES")

numeric_cols = [
    "Pre-Stress Traditional VaR", "Pre-Stress HHS VaR", "Pre-Stress VaR Diff (%)",
    "Pre-Stress Traditional ES", "Pre-Stress HHS ES", "Pre-Stress ES Diff (%)",
    "Stress Traditional VaR", "Stress HHS VaR", "Stress VaR Diff (%)",
    "Stress Traditional ES", "Stress HHS ES", "Stress ES Diff (%)"
]

comparison_df_styled = comparison_df.style.format(
    {col: "{:.2%}" for col in numeric_cols}
)

comparison_df_styled


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model

# Generate synthetic data for demonstration
np.random.seed(42)
days = 2000
dates = pd.date_range(start="2005-01-01", periods=days, freq="B")  # Business days
returns = np.random.normal(0, 0.01, size=days)

# Simulate a stress period with increased volatility (e.g., 2008 crisis)
stress_start = 800
stress_end = 1200
returns[stress_start:stress_end] += np.random.normal(0, 0.05, size=(stress_end - stress_start))

# Create a DataFrame
data = pd.DataFrame({"Return": returns}, index=dates)

# Split into pre-stress and stress periods
pre_stress_data = data[:stress_start]
stress_data = data[stress_start:stress_end]

# Fit a GARCH(1,1) model on pre-stress data
model = arch_model(pre_stress_data["Return"], vol="Garch", p=1, q=1)
fitted_model = model.fit(disp="off")

# Forecast volatility for the stress period
forecast_horizon = len(stress_data)
forecast = fitted_model.forecast(horizon=forecast_horizon, reindex=False)

# Extract forecasted and actual volatility
forecasted_volatility = np.sqrt(forecast.variance.values[-forecast_horizon:])
realized_volatility = stress_data["Return"].rolling(window=5).std()  # Rolling realized volatility

# Align indices for comparison
forecasted_volatility = pd.Series(forecasted_volatility, index=stress_data.index)

# Plot the forecasted and realized volatility
plt.figure(figsize=(14, 7))
plt.plot(forecasted_volatility, label="Forecasted Volatility (Pre-Stress Model)", color="blue")
plt.plot(realized_volatility, label="Realized Volatility (Stress Period)", color="red", alpha=0.7)
plt.axvspan(stress_data.index[0], stress_data.index[-1], color="orange", alpha=0.1, label="Stress Period")
plt.title("Volatility Forecast vs Realized Volatility", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Volatility", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
